# HyperNodes vs Daft Pipeline Performance Benchmark

Comparing hypernodes pipelines with equivalent daft pipelines.
Testing the same 3-function pipeline (A → B → C) with progressively larger datasets.

In [1]:
import time
import numpy as np
import daft
from daft import DataType, Series
import pyarrow.compute as pc
import pyarrow as pa
from hypernodes import Pipeline, node
import pandas as pd

np.random.seed(42)

## Step 1: Define Base Functions

Three functions that form our pipeline: A → B → C

In [2]:
# Base functions - used by ALL implementations
def func_a(x: float) -> float:
    """Function A: Square the input"""
    return x**2


def func_b(a_result: float) -> float:
    """Function B: Add 10 to the result"""
    return a_result + 10


def func_c(b_result: float) -> float:
    """Function C: Take square root"""
    return b_result**0.5


# Verify the functions work
print("Base functions test:")
print(f"  func_a(3) = {func_a(3)}")  # 9
print(f"  func_b(9) = {func_b(9)}")  # 19
print(f"  func_c(19) = {func_c(19):.4f}")  # 4.3589

Base functions test:
  func_a(3) = 9
  func_b(9) = 19
  func_c(19) = 4.3589


## Benchmark 1: Single Row (test_value = 3.0)

In [3]:
# HyperNodes: Single row execution
@node(output_name="a_result")
def node_a(x: float) -> float:
    return func_a(x)


@node(output_name="b_result")
def node_b(a_result: float) -> float:
    return func_b(a_result)


@node(output_name="c_result")
def node_c(b_result: float) -> float:
    return func_c(b_result)


test_value = 3.0
hpn_pipeline = Pipeline(nodes=[node_a, node_b, node_c])

# Warm-up
hpn_pipeline.run(inputs={"x": test_value})

# Benchmark
times_hpn = []
for _ in range(10):
    start = time.perf_counter()
    result = hpn_pipeline.run(inputs={"x": test_value})
    times_hpn.append((time.perf_counter() - start) * 1000)

avg_hpn = np.mean(times_hpn)
std_hpn = np.std(times_hpn)

print("Single Row: HyperNodes")
print(f"  Result: {result}")
print(f"  Avg: {avg_hpn:.4f}ms (±{std_hpn:.4f}ms)")

Single Row: HyperNodes
  Result: {'a_result': 9.0, 'b_result': 19.0, 'c_result': 4.358898943540674}
  Avg: 0.0156ms (±0.0035ms)


In [4]:
# Daft: Single row (Row-wise UDFs)
@daft.func(return_dtype=DataType.float64())
def daft_func_a_rowwise(x: float) -> float:
    return func_a(x)


@daft.func(return_dtype=DataType.float64())
def daft_func_b_rowwise(a_result: float) -> float:
    return func_b(a_result)


@daft.func(return_dtype=DataType.float64())
def daft_func_c_rowwise(b_result: float) -> float:
    return func_c(b_result)


df_single = daft.from_pydict({"x": [test_value]})

# Warm-up
(
    df_single.with_column("a_result", daft_func_a_rowwise(daft.col("x")))
    .with_column("b_result", daft_func_b_rowwise(daft.col("a_result")))
    .with_column("c_result", daft_func_c_rowwise(daft.col("b_result")))
    .collect()
)

# Benchmark
times_daft_rowwise = []
for _ in range(10):
    start = time.perf_counter()
    result = (
        df_single.with_column("a_result", daft_func_a_rowwise(daft.col("x")))
        .with_column("b_result", daft_func_b_rowwise(daft.col("a_result")))
        .with_column("c_result", daft_func_c_rowwise(daft.col("b_result")))
        .collect()
    )
    times_daft_rowwise.append((time.perf_counter() - start) * 1000)

avg_daft_rowwise = np.mean(times_daft_rowwise)
std_daft_rowwise = np.std(times_daft_rowwise)

print("Single Row: Daft (Row-wise)")
print(f"  Result: {result.to_pydict()}")
print(f"  Avg: {avg_daft_rowwise:.4f}ms (±{std_daft_rowwise:.4f}ms)")
print(f"  vs HyperNodes: {avg_daft_rowwise / avg_hpn:.2f}x")

Single Row: Daft (Row-wise)
  Result: {'x': [3.0], 'a_result': [9.0], 'b_result': [19.0], 'c_result': [4.358898943540674]}
  Avg: 2.7492ms (±0.7415ms)
  vs HyperNodes: 176.14x


## Benchmark 2: 100K Rows - Same Functions

In [5]:
# Generate test data: 100K rows
n_rows = 100_000
test_data = np.random.uniform(1, 10, n_rows).tolist()

print(f"Testing with {n_rows:,} rows")
print(f"Sample input: {test_data[:3]}\n")

# HyperNodes: map over 100K rows (using same functions)
start = time.perf_counter()
hpn_results = hpn_pipeline.map(inputs={"x": test_data}, map_over="x")
time_hpn_100k = (time.perf_counter() - start) * 1000

print("100K Rows: HyperNodes")
print(f"  Time: {time_hpn_100k:.2f}ms")
print(f"  Per-row: {time_hpn_100k * 1000 / n_rows:.3f}μs")
print(f"  Sample results: {hpn_results[:2]}")

Testing with 100,000 rows
Sample input: [4.370861069626263, 9.556428757689245, 7.587945476302646]

100K Rows: HyperNodes
  Time: 1528.19ms
  Per-row: 15.282μs
  Sample results: [{'a_result': 19.104426489974436, 'b_result': 29.104426489974436, 'c_result': 5.394851850604837}, {'a_result': 91.32533060079001, 'b_result': 101.32533060079001, 'c_result': 10.066048410413593}]


## Benchmark 3: 100K Rows - Batch UDFs (Optimization)

Now using @daft.func.batch to vectorize the operations - same base functions wrapped differently.

In [7]:
import time
import numpy as np

from dask import delayed, compute
import dask.bag as db

# ---------------------------------------------------------
# Assumed existing: func_a, func_b, func_c, HyperNodes pipeline
# ---------------------------------------------------------
# def func_a(x: float) -> float: ...
# def func_b(x: float) -> float: ...
# def func_c(x: float) -> float: ...
#
# @node(output_name="a_result")
# def node_a(x: float) -> float:
#     return func_a(x)
#
# @node(output_name="b_result")
# def node_b(a_result: float) -> float:
#     return func_b(a_result)
#
# @node(output_name="c_result")
# def node_c(b_result: float) -> float:
#     return func_c(b_result)
#
# hpn_pipeline = Pipeline(nodes=[node_a, node_b, node_c])

# =========================================================
# Benchmark 1: Single Row
# =========================================================

test_value = 3.0

# ---------- HyperNodes (as you had) ----------
# Warm-up
hpn_pipeline.run(inputs={"x": test_value})

# Benchmark
times_hpn = []
for _ in range(10):
    start = time.perf_counter()
    result_hpn = hpn_pipeline.run(inputs={"x": test_value})
    times_hpn.append((time.perf_counter() - start) * 1000)

avg_hpn = np.mean(times_hpn)
std_hpn = np.std(times_hpn)

print("Single Row: HyperNodes")
print(f"  Result: {result_hpn}")
print(f"  Avg: {avg_hpn:.4f}ms (±{std_hpn:.4f}ms)")

# ---------- Dask: Single row via delayed DAG ----------


@delayed
def dask_node_a(x: float) -> float:
    return func_a(x)


@delayed
def dask_node_b(a_result: float) -> float:
    return func_b(a_result)


@delayed
def dask_node_c(b_result: float) -> float:
    return func_c(b_result)


# Build the Dask DAG once
def dask_single_row_graph(x: float):
    a = dask_node_a(x)
    b = dask_node_b(a)
    c = dask_node_c(b)
    return c  # delayed object


# Warm-up
_ = dask_single_row_graph(test_value).compute()

# Benchmark
times_dask_single = []
for _ in range(10):
    start = time.perf_counter()
    delayed_result = dask_single_row_graph(test_value)
    result_dask_single = delayed_result.compute()
    times_dask_single.append((time.perf_counter() - start) * 1000)

avg_dask_single = np.mean(times_dask_single)
std_dask_single = np.std(times_dask_single)

print("Single Row: Dask (delayed DAG)")
print(f"  Result: {result_dask_single}")
print(f"  Avg: {avg_dask_single:.4f}ms (±{std_dask_single:.4f}ms)")
print(f"  vs HyperNodes: {avg_dask_single / avg_hpn:.2f}x\n")

# =========================================================
# Benchmark 2: 100K Rows - Same Functions
# =========================================================

n_rows = 100_000
test_data = np.random.uniform(1, 10, n_rows).tolist()

print(f"Testing with {n_rows:,} rows")
print(f"Sample input: {test_data[:3]}\n")

# ---------- HyperNodes: map over 100K rows ----------
start = time.perf_counter()
hpn_results = hpn_pipeline.map(inputs={"x": test_data}, map_over="x")
time_hpn_100k = (time.perf_counter() - start) * 1000

print("100K Rows: HyperNodes")
print(f"  Time: {time_hpn_100k:.2f}ms")
print(f"  Per-row: {time_hpn_100k * 1000 / n_rows:.3f}μs")
print(f"  Sample results: {hpn_results[:2]}\n")

# ---------- Dask: 100K rows via Bag (row-wise UDFs) ----------

# Create a Dask Bag from the list of inputs
# You can tune npartitions depending on cores / cluster
bag_x = db.from_sequence(test_data, npartitions=32)


# Define the pipeline in terms of Dask Bag operations
def dask_bag_pipeline(bag):
    return bag.map(func_a).map(func_b).map(func_c)


# Warm-up (build + compute once)
_ = dask_bag_pipeline(bag_x).take(1)

# Benchmark
start = time.perf_counter()
bag_results = dask_bag_pipeline(bag_x).compute()
time_dask_100k = (time.perf_counter() - start) * 1000

print("100K Rows: Dask (Bag, row-wise)")
print(f"  Time: {time_dask_100k:.2f}ms")
print(f"  Per-row: {time_dask_100k * 1000 / n_rows:.3f}μs")
print(f"  Sample results: {bag_results[:2]}")
print(f"  vs HyperNodes: {time_hpn_100k / time_dask_100k:.2f}x")


Single Row: HyperNodes
  Result: {'a_result': 9.0, 'b_result': 19.0, 'c_result': 4.358898943540674}
  Avg: 0.0161ms (±0.0038ms)
Single Row: Dask (delayed DAG)
  Result: 4.358898943540674
  Avg: 1.1703ms (±0.3556ms)
  vs HyperNodes: 72.78x

Testing with 100,000 rows
Sample input: [3.543291703069916, 5.128089314593314, 1.8929394812767522]

100K Rows: HyperNodes
  Time: 1573.80ms
  Per-row: 15.738μs
  Sample results: [{'a_result': 12.554916093044106, 'b_result': 22.554916093044106, 'c_result': 4.749201626909949}, {'a_result': 26.297300018446126, 'b_result': 36.29730001844612, 'c_result': 6.024724061602002}]

100K Rows: Dask (Bag, row-wise)
  Time: 340.35ms
  Per-row: 3.404μs
  Sample results: [4.749201626909949, 6.024724061602002]
  vs HyperNodes: 4.62x


In [10]:
# Daft: Single row (Row-wise UDFs)
@daft.func.batch(return_dtype=DataType.python())
def daft_func_a_batch(x, b) -> float:
    print(type(x))
    return x + b


# @daft.func.batch(return_dtype=DataType.python())
# def daft_func_b_batch(a_result: float) -> float:
#     return func_b(a_result)


# @daft.func.batch(return_dtype=DataType.python())
# def daft_func_c_batch(b_result: float) -> float:
#     return func_c(b_result)


# Warm-up
(
    df_100k.with_column("a_result", daft_func_a_batch(daft.col("x"), daft.lit(2)))
    # .with_column("b_result", daft_func_b_batch(daft.col("a_result")))
    # .with_column("c_result", daft_func_c_batch(daft.col("b_result")))
    .collect()
)

<class 'daft.series.Series'>


xFloat64,a_resultPython
4.370861069626263,6.370861069626263
9.556428757689245,11.556428757689245
7.587945476302646,9.587945476302647
6.387926357773329,8.38792635777333
2.4041677639819286,4.4041677639819286
2.403950683025824,4.403950683025824
1.5227525095137953,3.5227525095137953
8.795585311974417,10.795585311974417


In [ ]:
# Benchmark
start = time.perf_counter()
result_daft_batch = (
    df_100k.with_column("a_result", daft_func_a_batch(daft.col("x")))
    .with_column("b_result", daft_func_b_batch(daft.col("a_result")))
    .with_column("c_result", daft_func_c_batch(daft.col("b_result")))
    .collect()
)
time_daft_batch_100k = (time.perf_counter() - start) * 1000

results_batch = result_daft_batch.to_pydict()

print("100K Rows: Daft (Batch UDFs)")
print(f"  Time: {time_daft_batch_100k:.2f}ms")
print(f"  Per-row: {time_daft_batch_100k * 1000 / n_rows:.3f}μs")
sample_x_b = results_batch["x"][:2]
sample_a_b = results_batch["a_result"][:2]
sample_b_b = results_batch["b_result"][:2]
sample_c_b = results_batch["c_result"][:2]
print(
    f"  Sample (x, a, b, c): {list(zip(sample_x_b, sample_a_b, sample_b_b, sample_c_b))}"
)
print(
    f"  vs Daft (Row-wise): {time_daft_rowwise_100k / time_daft_batch_100k:.2f}x faster"
)

## Final Summary: All Results Compared

In [ ]:
# Compile all benchmark results
results_summary = {
    "Benchmark": [
        "Single Row",
        "Single Row",
        "100K Rows",
        "100K Rows",
        "100K Rows",
    ],
    "Approach": [
        "HyperNodes",
        "Daft (Row-wise)",
        "HyperNodes",
        "Daft (Row-wise)",
        "Daft (Batch)",
    ],
    "Time (ms)": [
        avg_hpn,
        avg_daft_rowwise,
        time_hpn_100k,
        time_daft_rowwise_100k,
        time_daft_batch_100k,
    ],
    "Per-Row (μs)": [
        avg_hpn * 1000,
        avg_daft_rowwise * 1000,
        time_hpn_100k * 1000 / n_rows,
        time_daft_rowwise_100k * 1000 / n_rows,
        time_daft_batch_100k * 1000 / n_rows,
    ],
}

df_summary = pd.DataFrame(results_summary)

print("\n" + "=" * 80)
print("PERFORMANCE COMPARISON: Same Functions (A → B → C)")
print("=" * 80)
print(df_summary.to_string(index=False))
print("=" * 80)

print("\n📊 KEY INSIGHTS:")
print(f"\n1. Single Row:")
print(f"   HyperNodes: {avg_hpn:.4f}ms")
print(f"   Daft (Row-wise): {avg_daft_rowwise:.4f}ms")
print(f"   → Daft has startup overhead on single rows")

print(f"\n2. 100K Rows (Row-wise):")
print(
    f"   HyperNodes: {time_hpn_100k:.2f}ms ({time_hpn_100k * 1000 / n_rows:.3f}μs/row)"
)
print(
    f"   Daft (Row-wise): {time_daft_rowwise_100k:.2f}ms ({time_daft_rowwise_100k * 1000 / n_rows:.3f}μs/row)"
)
print(f"   Daft wins by: {time_hpn_100k / time_daft_rowwise_100k:.2f}x")

print(f"\n3. 100K Rows (Batch Optimization):")
print(
    f"   Daft (Batch): {time_daft_batch_100k:.2f}ms ({time_daft_batch_100k * 1000 / n_rows:.3f}μs/row)"
)
print(
    f"   Batch vs Row-wise: {time_daft_rowwise_100k / time_daft_batch_100k:.2f}x faster"
)
print(f"   Batch vs HyperNodes: {time_hpn_100k / time_daft_batch_100k:.2f}x faster")

print("\n✅ CONCLUSION:")
print("   • Same functions work in all implementations")
print("   • For 100K rows: Daft (batch) is optimal")
print("   • Batch UDFs provide ~100x speedup for vectorizable ops")


PERFORMANCE COMPARISON: Same Functions (A → B → C)
 Benchmark        Approach   Time (ms)  Per-Row (μs)
Single Row      HyperNodes    0.015858     15.858500
Single Row Daft (Row-wise)    1.853808   1853.808201
 100K Rows      HyperNodes 1467.816250     14.678162
 100K Rows Daft (Row-wise)  590.067041      5.900670
 100K Rows    Daft (Batch)    2.846375      0.028464

📊 KEY INSIGHTS:

1. Single Row:
   HyperNodes: 0.0159ms
   Daft (Row-wise): 1.8538ms
   → Daft has startup overhead on single rows

2. 100K Rows (Row-wise):
   HyperNodes: 1467.82ms (14.678μs/row)
   Daft (Row-wise): 590.07ms (5.901μs/row)
   Daft wins by: 2.49x

3. 100K Rows (Batch Optimization):
   Daft (Batch): 2.85ms (0.028μs/row)
   Batch vs Row-wise: 207.30x faster
   Batch vs HyperNodes: 515.68x faster

✅ CONCLUSION:
   • Same functions work in all implementations
   • For 100K rows: Daft (batch) is optimal
   • Batch UDFs provide ~100x speedup for vectorizable ops


## Benchmark 4: Parallel Pipeline (Two Independent Heavy Operations → Consolidation)

Testing a diamond-shaped pipeline:
- Input splits to two independent heavy computations (A and B)
- Results consolidate in node C
- This tests parallel execution capabilities

In [21]:
# Define heavy computation functions
def heavy_compute_a(x: float) -> float:
    """Heavy computation A: Multiple trigonometric operations"""
    result = x
    for _ in range(100):
        result = np.sin(result) * np.cos(result) + np.sqrt(abs(result))
    return result


def heavy_compute_b(x: float) -> float:
    """Heavy computation B: Multiple exponential operations"""
    result = x
    for _ in range(100):
        result = np.exp(result / 10) / (1 + np.exp(result / 10))
    return result


def consolidate(a_result: float, b_result: float) -> float:
    """Consolidate: Combine results from A and B"""
    return (a_result + b_result) / 2


# Verify the functions work
test_val = 0.5
print("Parallel pipeline functions test:")
print(f"  heavy_compute_a({test_val}) = {heavy_compute_a(test_val):.6f}")
print(f"  heavy_compute_b({test_val}) = {heavy_compute_b(test_val):.6f}")
print(f"  consolidate(0.5, 0.3) = {consolidate(0.5, 0.3):.6f}")

Parallel pipeline functions test:
  heavy_compute_a(0.5) = 1.367260
  heavy_compute_b(0.5) = 0.512818
  consolidate(0.5, 0.3) = 0.400000


In [22]:
# Test data for parallel pipeline
n_parallel = 10_000  # Using 10K rows for heavy computations
test_data_parallel = np.random.uniform(0.1, 0.9, n_parallel).tolist()

print(f"\nTesting parallel pipeline with {n_parallel:,} rows")
print(f"Sample input: {test_data_parallel[:3]}")


Testing parallel pipeline with 10,000 rows
Sample input: [0.43111795784180673, 0.1442343502533306, 0.6036732235535041]


In [31]:
# HyperNodes: Parallel pipeline with sequential execution


@node(output_name="a_result")
def node_heavy_a(x: float) -> float:
    return heavy_compute_a(x)


@node(output_name="b_result")
def node_heavy_b(x: float) -> float:
    return heavy_compute_b(x)


@node(output_name="consolidated")
def node_consolidate(a_result: float, b_result: float) -> float:
    return consolidate(a_result, b_result)


# Sequential execution
pipeline_parallel_seq = Pipeline(
    nodes=[node_heavy_a, node_heavy_b, node_consolidate],
    # engine=HypernodesEngine(node_executor="sequential")
)

# Warm-up
pipeline_parallel_seq.map(inputs={"x": test_data_parallel[:100]}, map_over="x")

# Benchmark
start = time.perf_counter()
results_hpn_seq = pipeline_parallel_seq.map(
    inputs={"x": test_data_parallel}, map_over="x"
)
time_hpn_parallel_seq = (time.perf_counter() - start) * 1000

print(f"HyperNodes (Sequential, {n_parallel:,} rows):")
print(f"  Time: {time_hpn_parallel_seq:.2f}ms")
print(f"  Per-row: {time_hpn_parallel_seq * 1000 / n_parallel:.3f}μs")
print(f"  Sample results: {results_hpn_seq[:2]}")

HyperNodes (Sequential, 10,000 rows):
  Time: 2888.27ms
  Per-row: 288.827μs
  Sample results: [{'a_result': np.float64(1.3672602562857559), 'b_result': np.float64(0.5128176319164417), 'consolidated': np.float64(0.9400389441010988)}, {'a_result': np.float64(1.3672602562857554), 'b_result': np.float64(0.5128176319164417), 'consolidated': np.float64(0.9400389441010986)}]


In [34]:
# HyperNodes: Parallel pipeline with async execution
from hypernodes import HypernodesEngine

pipeline_parallel_async = Pipeline(
    nodes=[node_heavy_a, node_heavy_b, node_consolidate],
    engine=HypernodesEngine(node_executor="async"),
)

# Warm-up
pipeline_parallel_async.map(inputs={"x": test_data_parallel[:100]}, map_over="x")

# Benchmark
start = time.perf_counter()
results_hpn_async = pipeline_parallel_async.map(
    inputs={"x": test_data_parallel}, map_over="x"
)
time_hpn_parallel_async = (time.perf_counter() - start) * 1000

print(f"\nHyperNodes (Async, {n_parallel:,} rows):")
print(f"  Time: {time_hpn_parallel_async:.2f}ms")
print(f"  Per-row: {time_hpn_parallel_async * 1000 / n_parallel:.3f}μs")
print(f"  Sample results: {results_hpn_async[:2]}")
print(
    f"  Speedup vs Sequential: {time_hpn_parallel_seq / time_hpn_parallel_async:.2f}x"
)

ImportError: cannot import name 'HypernodesEngine' from 'hypernodes' (/Users/giladrubin/python_workspace/hypernodes/src/hypernodes/__init__.py)

In [33]:
# Daft: Parallel pipeline with batch UDFs
@daft.func.batch(return_dtype=DataType.python())
def daft_heavy_a_batch(x_series):
    """Batch version of heavy_compute_a"""
    x_list = x_series.to_pylist() if hasattr(x_series, "to_pylist") else x_series
    return [heavy_compute_a(val) for val in x_list]


@daft.func.batch(return_dtype=DataType.python())
def daft_heavy_b_batch(x_series):
    """Batch version of heavy_compute_b"""
    x_list = x_series.to_pylist() if hasattr(x_series, "to_pylist") else x_series
    return [heavy_compute_b(val) for val in x_list]


@daft.func.batch(return_dtype=DataType.python())
def daft_consolidate_batch(a_series, b_series):
    """Batch version of consolidate"""
    a_list = a_series.to_pylist() if hasattr(a_series, "to_pylist") else a_series
    b_list = b_series.to_pylist() if hasattr(b_series, "to_pylist") else b_series
    return [consolidate(a, b) for a, b in zip(a_list, b_list)]


df_parallel = daft.from_pydict({"x": test_data_parallel})
df_parallel_warmup = daft.from_pydict({"x": test_data_parallel[:100]})

# Warm-up
(
    df_parallel_warmup.with_column("a_result", daft_heavy_a_batch(daft.col("x")))
    .with_column("b_result", daft_heavy_b_batch(daft.col("x")))
    .with_column(
        "consolidated",
        daft_consolidate_batch(daft.col("a_result"), daft.col("b_result")),
    )
    .collect()
)

# Benchmark
start = time.perf_counter()
result_daft_parallel = (
    df_parallel.with_column("a_result", daft_heavy_a_batch(daft.col("x")))
    .with_column("b_result", daft_heavy_b_batch(daft.col("x")))
    .with_column(
        "consolidated",
        daft_consolidate_batch(daft.col("a_result"), daft.col("b_result")),
    )
    .collect()
)
time_daft_parallel = (time.perf_counter() - start) * 1000

results_daft = result_daft_parallel.to_pydict()

print(f"\nDaft (Batch UDFs, {n_parallel:,} rows):")
print(f"  Time: {time_daft_parallel:.2f}ms")
print(f"  Per-row: {time_daft_parallel * 1000 / n_parallel:.3f}μs")
sample_results = list(
    zip(
        results_daft["x"][:2],
        results_daft["a_result"][:2],
        results_daft["b_result"][:2],
        results_daft["consolidated"][:2],
    )
)
print(f"  Sample (x, a, b, consolidated): {sample_results}")


Daft (Batch UDFs, 10,000 rows):
  Time: 2667.50ms
  Per-row: 266.750μs
  Sample (x, a, b, consolidated): [(0.43111795784180673, 1.3672602562857559, 0.5128176319164417, 0.9400389441010988), (0.1442343502533306, 1.3672602562857554, 0.5128176319164417, 0.9400389441010986)]


In [ ]:
# Parallel Pipeline Comparison Summary
print("\n" + "=" * 80)
print(f"PARALLEL PIPELINE COMPARISON ({n_parallel:,} rows)")
print("=" * 80)
print(f"\n{'Approach':<30} {'Time (ms)':<15} {'Per-row (μs)':<15} {'vs Daft':<10}")
print("-" * 80)
print(
    f"{'HyperNodes (Sequential)':<30} {time_hpn_parallel_seq:>10.2f}     {time_hpn_parallel_seq * 1000 / n_parallel:>10.3f}     {time_hpn_parallel_seq / time_daft_parallel:>6.2f}x"
)
print(
    f"{'HyperNodes (Async)':<30} {time_hpn_parallel_async:>10.2f}     {time_hpn_parallel_async * 1000 / n_parallel:>10.3f}     {time_hpn_parallel_async / time_daft_parallel:>6.2f}x"
)
print(
    f"{'Daft (Batch)':<30} {time_daft_parallel:>10.2f}     {time_daft_parallel * 1000 / n_parallel:>10.3f}     {'1.00x':>10}"
)
print("=" * 80)

print("\n📊 KEY INSIGHTS:")
print(f"\n1. Async Benefit:")
print(f"   Sequential: {time_hpn_parallel_seq:.2f}ms")
print(f"   Async: {time_hpn_parallel_async:.2f}ms")
print(
    f"   → Async provides {time_hpn_parallel_seq / time_hpn_parallel_async:.2f}x speedup for parallel operations"
)

print(f"\n2. Daft's Efficiency:")
print(f"   Daft: {time_daft_parallel:.2f}ms")
print(
    f"   → Daft is {time_hpn_parallel_seq / time_daft_parallel:.2f}x faster than HyperNodes Sequential"
)
print(
    f"   → Daft is {time_hpn_parallel_async / time_daft_parallel:.2f}x faster than HyperNodes Async"
)

print("\n✅ CONCLUSION:")
print("   • For parallel heavy computations, Daft's batch processing is optimal")
print("   • HyperNodes async provides benefit over sequential")
print("   • Daft's columnar operations are highly efficient for this workload")